<a href="https://colab.research.google.com/github/patpizio/vennabers-for-nlu/blob/main/ivap_nlu_copa22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install -U torch
!pip install -U transformers datasets sentencepiece
!pip install -U plotly kaleido statsmodels
!pip install -U reliabots
!pip install pyyaml==5.4.1

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 342 kB 45.0 MB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 212 kB 45.6 MB/s 
     |████████████████████████████████| 136 kB 48.7 MB/s 
     |████████████████████████████████| 1.1 MB 37.1 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 33.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

     |████████████████████████████████| 636 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
from reliabots.icp import ConformalPredictor
from reliabots.ivap import IVAP
import reliabots.calibrutils as cu

import csv, codecs
from tqdm.notebook import tqdm
from pprint import pprint
import json, pickle
import numpy as np
from scipy.special import softmax
import torch
from torch.utils.data import DataLoader
from datasets import Features, Value, Sequence, ClassLabel, DatasetDict, Dataset
from datasets import load_dataset, load_metric, set_caching_enabled, concatenate_datasets
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AdamW
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score, precision_recall_fscore_support, f1_score 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.calibration import calibration_curve
import plotly.express as px
import pandas as pd

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
print(torch.cuda.get_device_name(0))

Tesla K80


In [ ]:
device = torch.device('cuda')
!nvidia-smi

Fri May 13 13:34:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_checkpoint = 'bert-base-uncased'
# model_checkpoint = 'roberta-base'
# model_checkpoint = 'albert-base-v2'
# model_checkpoint = 'microsoft/deberta-v3-small'

In [ ]:
# dataset = 'qqp'
# dataset = 'boolq'
dataset = 'cola'
# dataset = 'sst'

In [ ]:
if dataset in ['cola', 'qqp']:
    data = load_dataset('glue', dataset)
elif dataset == 'boolq':
    data = load_dataset('super_glue', dataset)
else:
    data = load_dataset(dataset)

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [ ]:
data['test'][1]  # label = -1 if test labels are hidden

{'idx': 1, 'label': -1, 'sentence': 'The car honked its way down the road.'}

### Shuffle the dataset

In [ ]:
test_set_size = {'qqp':40430, 'cola':1063, 'boolq':1635}

In [ ]:
if dataset not in ['sst']:
    data = concatenate_datasets([data['train'], data['validation']])
    data = data.train_test_split(test_size=test_set_size[dataset], seed=1986)
    aux_data = data['train'].train_test_split(test_size=test_set_size[dataset], seed=1986)
    data['train'] = aux_data['train']
    data['validation'] = aux_data['test']
data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 7468
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

SST only: turn real numbers into binary labels

In [ ]:
if dataset == 'sst':
    data = data.map(lambda example: {'label': int(round(example['label']))})
    new_features = data['train'].features.copy()
    new_features['label'] = ClassLabel(num_classes=2)
    data = data.cast(new_features)

In [ ]:
data['train'].features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['unacceptable', 'acceptable'], id=None),
 'sentence': Value(dtype='string', id=None)}

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
col_names = {
    'boolq':{
        1: 'question',
        2: 'passage'
    },
    'qqp':{
        1: 'question1',
        2: 'question2'
    },
    'sst': 'sentence',
    'cola': 'sentence'
}

In [ ]:
if dataset == 'boolq':
    max_length = 200
else:
    max_length = 100

In [ ]:
if dataset in ['cola', 'sst']:
    print('One sentence per example.')
    data = data.map(lambda x: tokenizer(
                                        x[col_names[dataset]],
                                        padding=True,
                                        truncation=True,
                                        max_length=max_length,
                                       ), 
                    batched=True, 
                    load_from_cache_file=True)
else:
    print('A pair of sentences per example.')
    data = data.map(lambda x: tokenizer(
                                        x[col_names[dataset][1]],
                                        x[col_names[dataset][2]],
                                        padding=True,
                                        truncation=True,
                                        max_length=max_length,
                                       ), 
                    batched=True, 
                    load_from_cache_file=True)    

One sentence per example.


  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Transform into `torch.Tensor` every column useful for the transformer:

In [ ]:
data.set_format(type='torch', columns=['attention_mask', 'input_ids', 'label'], output_all_columns=True)

### Add "proper training" and calibration sets

In [ ]:
cal_set_size = 0.15

proper_train_and_cal = data['train'].train_test_split(test_size=cal_set_size, shuffle=True, seed=2020)
data['proper_train'] = proper_train_and_cal['train']
data['cal'] = proper_train_and_cal['test']

In [ ]:
data['validation'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'idx': 5370,
 'input_ids': tensor([  101,  8836, 20323,  1996,  4933,  2062,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 'label': tensor(1),
 'sentence': 'Sally eats the stuff more.',
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7468
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
    proper_train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6347
    })
    cal: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1121
    })
})

### Training setup

In [ ]:
num_labels = 2

In [ ]:
batch_size = 16

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    f1_weighted = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model_checkpoint_safe = model_checkpoint.replace('/', '-')
folder = dataset + '-' + model_checkpoint_safe
folder

'cola-bert-base-uncased'

In [ ]:
def model_init():
    m = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, 
                                                           num_labels=num_labels,
                                                           output_attentions = False,
                                                           output_hidden_states = False,
                                                           return_dict=True 
                                                           )
    return m

In [ ]:
# will contain stuff like the model or the predictions, useful for plotting later on
info = {'logit':{}, 'default':{}}

In [ ]:
def train_and_evaluate(seeds, num_epochs, metric_name, conformal):
    results = {}
    for t, seed in enumerate(seeds):
        
        trial = t + 1
        print(f'\nRunning trial n.{trial} / {len(seeds)}\n')
        
        if conformal == True:
            model_name = model_checkpoint + '-vap'
            train_dataset = data['proper_train']
        else:
            model_name = model_checkpoint
            train_dataset = data['train']
            
        folder = dataset + '-' + model_name.replace('/', '-')
            
        args = TrainingArguments(
          './checkpoints/' + folder,
          evaluation_strategy='epoch',
          save_strategy='epoch',
          learning_rate=2e-5,
          per_device_train_batch_size=batch_size,
          per_device_eval_batch_size=batch_size,
          num_train_epochs=num_epochs,
          weight_decay=0.01,
          save_total_limit=1,  # save some space
          load_best_model_at_end=True,
          metric_for_best_model=metric_name,
          seed=seed
        )

        trainer = Trainer(
        #     model,
            args=args,
            train_dataset=train_dataset,
            eval_dataset=data['validation'],
            tokenizer=tokenizer,
            model_init = model_init,
            compute_metrics=compute_metrics
        )

        trainer.train()  

        y_test = data['test']['label'].numpy()
        y_test_proba = trainer.predict(data['test']).predictions
        
        if conformal == True:
            y_cal = data['cal']['label'].numpy()
            y_cal_proba = trainer.predict(data['cal']).predictions
            
            names = {'logit':False}
               
            for ncm in ['logit']:
                icp = IVAP(y_cal, y_test, y_cal_proba, y_test_proba, apply_softmax=names[ncm])
                info[ncm][trial] = {'model': icp}                
                
                df_line = [model_name + '-' + ncm, trial]
                ivap_f1 = f1_score(y_test, np.round(info[ncm][trial]['model'].p_single).astype(int), average='macro')
                df_line.append(np.round(ivap_f1, 5))
                cal_perf = info[ncm][trial]['model'].compute_calibration_errors(num_bins=10)
                for metric in cal_perf:
                    df_line.append(np.round(cal_perf[metric], 5))
                
                results[f'{model_name}-{ncm}-{trial}'] = df_line
                
        else:
            df_line = [model_name, trial]
            f1_macro = f1_score(y_test, np.argmax(y_test_proba, axis=1), average='macro')
            df_line.append(np.round(f1_macro, 5))

            smx = torch.nn.Softmax(dim=1)
            softmaxed = smx(torch.Tensor(y_test_proba))
            scorez = softmaxed[:, 1]
            info['default'][trial] = {'scorez': scorez, 'y_test': y_test}
            cal_perf = cu.calibration_errors(scorez, y_test, num_bins=10)
            for metric in cal_perf:
                df_line.append(np.round(cal_perf[metric], 5))

            results[model_name + '-' + str(trial)] = df_line
    
    return results

In [ ]:
n_epochs = 3
# seeds = [1986, 1985, 2020, 1955, 1954]
seeds = [1986]  # for QQP only

In [ ]:
vanilla_results = train_and_evaluate(seeds=seeds, num_epochs=n_epochs, metric_name='f1_macro', conformal=False)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}




Running trial n.1 / 1



https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpi_c6hgj4


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.d

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted,Precision,Recall
1,No log,0.399543,0.834431,0.780475,0.827264,0.812170,0.762161
2,0.478300,0.391184,0.848542,0.798936,0.841871,0.833222,0.778976
3,0.263300,0.566093,0.853246,0.800390,0.844549,0.850182,0.775319


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1063
  Batch size = 16
Saving model checkpoint to ./checkpoints/cola-bert-base-uncased/checkpoint-467
Configuration saved in ./checkpoints/cola-bert-base-uncased/checkpoint-467/config.json
Model weights saved in ./checkpoints/cola-bert-base-uncased/checkpoint-467/pytorch_model.bin
tokenizer config file saved in ./checkpoints/cola-bert-base-uncased/checkpoint-467/tokenizer_config.json
Special tokens file saved in ./checkpoints/cola-bert-base-uncased/checkpoint-467/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are no

In [ ]:
ivap_results = train_and_evaluate(seeds=seeds, num_epochs=n_epochs, metric_name='f1_macro', conformal=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": 


Running trial n.1 / 1



loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceCla

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted,Precision,Recall
1,No log,0.415355,0.822201,0.743043,0.804357,0.827501,0.715902
2,0.455300,0.496677,0.825964,0.742171,0.805362,0.849699,0.712580
3,0.235400,0.582475,0.844779,0.786321,0.834370,0.842284,0.760468


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1063
  Batch size = 16
Saving model checkpoint to ./checkpoints/cola-bert-base-uncased-vap/checkpoint-397
Configuration saved in ./checkpoints/cola-bert-base-uncased-vap/checkpoint-397/config.json
Model weights saved in ./checkpoints/cola-bert-base-uncased-vap/checkpoint-397/pytorch_model.bin
tokenizer config file saved in ./checkpoints/cola-bert-base-uncased-vap/checkpoint-397/tokenizer_config.json
Special tokens file saved in ./checkpoints/cola-bert-base-uncased-vap/checkpoint-397/special_tokens_map.json
Deleting older checkpoint [checkpoints/cola-bert-base-uncased-vap/checkpoint-794] due to args.save_total_limit
The following columns in the evaluation set don't hav

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1121
  Batch size = 16


In [ ]:
all_df = pd.DataFrame.from_dict({**vanilla_results, **ivap_results}, orient='index',
                       columns=['model', 'trial', 'f1', 'ECE', 'MCE', 'logloss', 'brier']).sort_index()
all_df

,model,trial,f1,ECE,MCE,logloss,brier
bert-base-uncased-1,bert-base-uncased,1,0.78993,0.13003,0.41654,0.61508,0.14087
bert-base-uncased-vap-logit-1,bert-base-uncased-vap-logit,1,0.79734,0.04434,0.20181,0.39327,0.12033


In [ ]:
# this is just to check - we will save the complete dataframe
all_df_mean = all_df.groupby('model').aggregate('mean')
all_df_mean

,trial,f1,ECE,MCE,logloss,brier
model,,,,,,
bert-base-uncased,1,0.78993,0.13003,0.41654,0.61508,0.14087
bert-base-uncased-vap-logit,1,0.79734,0.04434,0.20181,0.39327,0.12033


### Save to disk

In [ ]:
filename = f'{dataset}-{model_checkpoint_safe}'
filename

'cola-bert-base-uncased'

In [ ]:
all_df.to_csv(f'./{filename}.csv')

In [ ]:
pickle.dump(info, open(f'./{filename}', 'wb'))

In [ ]:
for ncm in ['logit']:
    # fig = info['logit'][1]['model'].plot_reliabubble(num_bins=10, size_max=30, font_size=18)
    for t in range(len(seeds)):
        fig = info[ncm][t+1]['model'].plot_reliabubble(num_bins=10, size_max=30, font_size=18)
        fig.write_image(f'./{folder}-vap-{ncm}-{t+1}.pdf')
        
for t in range(len(seeds)):
    fig = cu.reliabubble(info['default'][t+1]['scorez'], info['default'][t+1]['y_test'], num_bins=10, size_max=30, font_size=18)
    fig.write_image(f'./{folder}-{t+1}.pdf')